In [15]:
import time 
import urllib
import requests
import operator
import numpy as np
import json
from IPython.display import Image as ipImage, display
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline 

_url = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/a6fb26d9-01bc-4999-aea1-eb4bfb513148/url?'
_iterationId = '74c1f0c3-3095-4386-aba3-0fcd3e932fd8'
_key = '4b3de28d2ff44e90a02c49efee2e635c'

In [16]:
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [17]:
def prediction( url ):
    headers = dict()
    headers['Prediction-key'] = _key
    headers['Content-Type'] = 'application/json' 

    json = { 'url': url } 
    data = None

    params = urllib.parse.urlencode({
        'iterationId': _iterationId,
    })

    result = processRequest( json, data, headers, params )
    
    return result

In [18]:
urlImage = 'http://dadoteca.com.br/arquivos/m1.jpg'
img = ipImage(url=urlImage, width=400, height=400)
display(img)

p1 = prediction(urlImage)

In [19]:
print(json.dumps(p1, indent=2, sort_keys=True))

{
  "created": "2018-06-21T18:44:20.0743059Z",
  "id": "65b4bc5a-de25-493a-8b95-343b60d3a94d",
  "iteration": "74c1f0c3-3095-4386-aba3-0fcd3e932fd8",
  "predictions": [
    {
      "probability": 0.9979067,
      "tagId": "e3ed9cf5-600b-4ac2-9dbc-3e29bae5625c",
      "tagName": "Moto"
    },
    {
      "probability": 0.00133252947,
      "tagId": "c3505cfb-5e0e-42f6-9d63-93fc1ce58aaa",
      "tagName": "Aviao"
    },
    {
      "probability": 0.000919278653,
      "tagId": "b75a8e78-7de4-47c0-9c37-37ea12e015f0",
      "tagName": "Carro"
    }
  ],
  "project": "a6fb26d9-01bc-4999-aea1-eb4bfb513148"
}
